#第四次讀書會報告 -

組別:第9組

Date:2024/05/28

#1.載入環境及安裝套件

In [ ]:
#連接colab雲端資料夾
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/LLM') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['requirements.txt', 'dict', 'data']

安裝套件

In [ ]:
!pip install typer==0.12.3

In [ ]:
!pip install jieba

!pip install sentence-transformers
!pip install transformers
!pip install sentencepiece

# 用於 transformer 加速
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install langchain-chroma
!pip install pypdf
!pip install langchain
!pip install huggingface_hub
!pip install grandalf
!pip install pandas
!pip install ctransformers

Looking in indexes: https://pypi.org/simple/


In [ ]:
#此套件為爬取PDF內容
!pip install pypdf

In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.5 MB/s eta 0:00:00


載入套件

In [ ]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import BitsAndBytesConfig # huggingface 量化

from langchain_community.llms import HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

連結HuggingFace Token

In [ ]:
#連結HuggingFace Token
# from google.colab import userdata
# my_api_key = userdata.get('my_api_key')

#輸入HUGGINGFACE的API TOKEN
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


In [ ]:
quant_config = BitsAndBytesConfig(load_in_4bit=True)

print(quant_config.is_quantizable())
print(quant_config.quantization_method())

True
fp4


#2.載入 LLM 模型 & 基本詢問
此部分載入google/gemma-1.1-7b-it模型
參考網址：https://huggingface.co/google/gemma-1.1-7b-it

In [ ]:
model_id = "google/gemma-1.1-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HUGGINGFACEHUB_API_TOKEN)
# device_map="auto" 使用 GPU 加速
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quant_config, token=HUGGINGFACEHUB_API_TOKEN)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


##2.1 HuggingFaceEndpoint
- Langchian 整合 huggingface severless api (https://huggingface.co/docs/api-inference/index)
- 可使用超過 100,000 部署在 Hugging Face Hub 上的模型
- Host 在 Hugging Face shared infrastructure 上，無需本地資源

以下為導入模型後進行詢問測試

In [ ]:
# repo_id: HuggingFace 上的模型名稱，本範例使用 gemma 7b instruct 版本
# huggingface: https://huggingface.co/google/gemma-1.1-7b-it
# temperature: 模型回答的活潑程度，0~1 越大每次回答的多樣性越高

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-1.1-7b-it", temperature=1.0,model_kwargs={'token':HUGGINGFACEHUB_API_TOKEN, 'use_cache':False}
)
#詢問測試
llm.invoke("你好，請介紹一下HuggingFace？")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


'\n\n**HuggingFace** 是一個基於雲的平台，專門用於自然語言處理（NLP）和機器學習（ML）模型的開發、分發和運營。它提供了以下主要功能：\n\n* **模型儲存庫：**儲存和管理各種訓練好的NLP和ML模型。\n* **模型轉換器：**讓模型更易於使用，並在不同架構和程式語言之間進行轉換。\n* **模型運營：**提供即時模型運營和推斷服務。\n* **社區協作：**一個線上社群，提供模型分享、討論和協作。\n\n**HuggingFace 的優點：**\n\n* **易於使用：**平台提供簡潔易用的介面，並有大量的線上資源和說明。\n* **大規模的模型庫：**儲存著各種訓練好的模型，適合不同的NLP任務。\n* **跨平台相容：**模型可以輕鬆地轉換為不同架構和程式語言的版本。\n* **即時運營：**不需要訓練模型，就能立即使用預訓練模型進行推斷。\n\n**HuggingFace 的應用：**\n\n* 文本摘要和簡約\n* 情感分析和情緒偵測\n* 自然語言理解和翻譯\n* 文檔分類和主題建模\n* 機器翻譯和語音辨識\n\n**結論：**\n\nHuggingFace 是一個有價值的平台，用於開發、分發和運營 NLP 和 ML 模型。其易於使用、大規模的模型庫和即時運營功能使它成為各種 NLP 任務的理想解決方案。'

In [ ]:
llm.invoke("那在HuggingFace上有哪些模型是推薦的呢？")

'\n\n以下是一些推薦的模型：\n\n* **語言模型：**\n    * GPT-3\n    * Pegasus\n    * T5\n* **文本摘要：**\n    * Pegasus-Summary\n    * Transformers-3B-summarizer\n* **自然語言理解：**\n    * DistilBERT\n    * BERT\n    * RoBERTa\n* **情感分析：**\n    * Flair\n    * Text2emotion\n\n除了以上列出的模型之外，HuggingFace還有許多其他的模型可供使用。您可以根據自己的特定需求和目標進行選擇。以下是一些可以幫助您找到適合模型的資源：\n\n* **HuggingFace 模型中心：**https://huggingface.co/models\n* **HuggingFace 搜索引擎：**https://huggingface.co/search\n\n**注意事項：**\n\n* 不同的模型在效能和準確性方面有所不同。\n* 在使用模型之前，您需要了解其目標和應用場景。\n* 建議在使用模型之前對其進行測試和評估。'

以上兩個例子為本組測試模型導入成效,可以發現皆能回答出我們想要的答案

#3.Chain的建立與測試
- LCEL (LangChain Expression Language)
    - 官網介紹：https://python.langchain.com/v0.1/docs/expression_language/why/

    - Langchain 說明：LCEL 可以讓我們更簡單的開發複雜的 chain 應用，可以支援：Streaming, Parallelism, logging。

- Chain
    - 可重複執行
    - 可自由組合
    - 有兩種主要的型態
        1. RunnableSequence: 依序執行每個 Runnable
        2. RunnableParallel: 同時執行 Runnable

以下為RunnableSequence
依序執行每個 Runnable
使用 "|" 連結

In [ ]:
#“你是一位社群媒體分析專家”以此設定模型的角色,增強回答的準確度
template = """你是一位社群媒體分析專家，尤其對Dcard有獨到見解,請問以下Dcard文章句子中的情緒為何（正向,中性,負向）: {question}"""

prompt = PromptTemplate.from_template(template)

prompt_output = prompt.invoke({"question": '跟牡羊女分手後他還是願意回我,但很不耐煩,也不愛回,但會願意跟我見面,拿東西給我之類的,他之前的前任分手後他都會把對方封鎖'})
prompt_output

StringPromptValue(text='你是一位社群媒體分析專家，尤其對Dcard有獨到見解,請問以下Dcard文章句子中的情緒為何（正向,中性,負向）: 跟牡羊女分手後他還是願意回我,但很不耐煩,也不愛回,但會願意跟我見面,拿東西給我之類的,他之前的前任分手後他都會把對方封鎖')

接著將prompt_output結果丟入詢問,可以得到句子分段且各段的情緒值


In [ ]:
print(prompt_output)
llm_output = llm.invoke(prompt_output)
llm_output

text='你是一位社群媒體分析專家，尤其對Dcard有獨到見解,請問以下Dcard文章句子中的情緒為何（正向,中性,負向）: 跟牡羊女分手後他還是願意回我,但很不耐煩,也不愛回,但會願意跟我見面,拿東西給我之類的,他之前的前任分手後他都會把對方封鎖'


'，這一次也是這樣，但我並不喜歡這種不信任，因為我不覺得這樣有意義。\n\n**情緒分析：**\n\n**正向：**回我，拿東西給我之類的\n\n**中性：**很不耐煩,也不愛回\n\n**負向：**封鎖，不信任，沒有意義\n\n\n**修正：**\n\n**情緒分析：**\n\n**正向：**回我，拿東西給我之類的\n\n**中性：**很不耐煩,也不愛回\n\n**負向：**封鎖，不信任，沒有意義，有破裂關係的趨勢\n\n\n**說明：**\n\n* 文章中「回我，拿東西給我之類的」這段被認為是正向情緒，因為它表示了對方的回饋和幫助。\n* 「很不耐煩,也不愛回」這段被認為是中性情緒，因為它表示了對方的行為沒有特別的積極或消極情緒。\n* 「封鎖，不信任，沒有意義」這段被認為是負向情緒，因為它表示了對方的破裂關係趨勢。'

In [ ]:
# RunnablePassthrough: 可以不變的傳遞整個 input

template = """你是一位社群媒體分析專家，尤其對Dcard有獨到見解,也很擅長看出文章情緒,請幫以下Dcard文章句子評分情緒分數（介於 -1.0~1.0，越高分代表越正向，反之越負向）: {question}"""

prompt = PromptTemplate.from_template(template)

sentiment_value_chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm
)
#分析解釋情緒值原因
sentiment_value_chain.invoke("私底下我們也會聊天，他也會跟我抱怨事情，但！上學的時候 有時候秒回我 有時候又過了好幾個小時才回，雖然事後他都會跟我說他在幹嘛好奇獅子男私聊的時候都會轉移話題或避開話題嗎？")


'好奇獅子男私聊的時候會轉移話題或避開話題嗎？\n\n**評分：** -0.4\n\n**解釋：** 文章中並非明確表達負面情緒，但出現了對男性溝通方式的負面猜測，並表達對男性私聊內容的猜測，這將會導致負面情緒分數。'

以上部分則是可分析解釋丟入詢問的內容其情緒值及分析的原因

RunnableParallel測試: 同時執行 Runnable

In [ ]:
# 設定繁體中文詞庫
jieba.set_dictionary("/content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/LLM/dict/dict.txt.big")

# 新增stopwords
# jieba.analyse.set_stop_words('./dict/stop_words.txt') #jieba.analyse.extract_tags才會作用
with open("/content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/LLM/dict/stop_words.txt", encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

# 設定斷詞 function
def getToken(row):
    seg_list = jieba.cut(row, cut_all=False)
    seg_list = [
        w for w in seg_list if w not in stopWords and len(w) > 1
    ]  # 篩選掉停用字與字元數大於1的詞彙
    return seg_list

# 讀取情緒字典
liwc_dict = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/LLM/dict/liwc/LIWC_CH.csv")
liwc_dict = liwc_dict.rename(columns={'name': 'word', "class": 'sentiments'})

In [ ]:
# call lexicon base function,定義lexicon判斷情緒值的函數
def lexicon_base_sentiment_value(value: dict) -> str:
    tokens = getToken(value['question'])
    df = pd.DataFrame(tokens, columns=['word'])
    liwc_df = pd.merge(df, liwc_dict, how="left")
    sentiment_count = pd.DataFrame(
        liwc_df.groupby(["sentiments"]).size()
    ).reset_index()
    result_dict = sentiment_count.set_index('sentiments')[0].to_dict()
    print(result_dict)
    s_value = (result_dict.get('positive') if result_dict.get('positive') != None else 0) - (result_dict.get('negative') if result_dict.get('negative') != None else 0)
    return s_value

#形成chain
lexicon_sentiment_value_chain = RunnableLambda(lexicon_base_sentiment_value)

#將unnable_1,runnable_2引入
parallel = {"runnable_1": lexicon_sentiment_value_chain, "runnable_2": sentiment_value_chain}

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke({'question':'對付水瓶男只有一個宗旨,永遠不要讓他摸清楚你的底細,尤其是土象星座,利用你的未知去勾起水瓶男對你的好奇,當他好奇你想認識你時,會頻繁的約你出去,這時就要開始給他三成的實話,七成的含糊不清'})

print(answer) #得到情緒評分並解釋分析原因

Building prefix dict from /content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/LLM/dict/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/LLM/dict/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u656360c3f5289b439ca0eb3d165ba7fc.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.u656360c3f5289b439ca0eb3d165ba7fc.cache
Loading model cost 3.735 seconds.
DEBUG:jieba:Loading model cost 3.735 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


{'affect': 1}
{'runnable_1': 0, 'runnable_2': '。\n\n**情緒評分：** -0.3\n\n**評估：** 文章情緒為負向，主要表現為對水瓶男的控制和利用。作者建議在水瓶男好奇時提供混亂的資訊，以控制其對自己的認識。'}


In [ ]:
#同時執行runnable_1,runnable_2
chain = RunnableParallel(lexicon=lexicon_sentiment_value_chain, LLM=sentiment_value_chain)
chain.invoke({'question':'我自己會害怕交往後會不會要常常為了訊息的事吵架,這件事我有跟他說過我很在乎,所以我們算是達成共識再相處看看'})

{'affect': 2, 'anx': 1, 'negative': 1}


{'lexicon': -1,
 'LLM': '\n\n**情感分數：**-0.2\n\n**解釋：**文章中表達的是作者對交往後可能需要頻繁溝通的事感到困惑和擔憂，但同時也表達了對彼此的理解和共識。這個情緒混合，略帶負面，但並非極度負面。'}

列出不同情緒值的數量，如以上{'affect': 2, 'anx': 1, 'negative': 1}並得到LLM情緒評分值及解釋原因

#4.資料集實作 Information Extraction (Tagging)
https://python.langchain.com/docs/use_cases/tagging/

##4.1資料蒐集
本小組自Tarflow下載Dacrd上4個個版的文章資料集，並執行程式合成一個資料集，共3,598筆資料。

其內容包含system_id、ataSource、artUrl、artDate、artTitle、artContent、boardID(文章所屬個版)、
commentCount(評論次數)、department、gender、school等欄位。

Data includes:
*   Dcard文章 from 2024/03/01 - 2024/03/25
*   專版包含以下4板：
*   穿搭版
*   星座版
*   理財版
*   研究所版

資料來源:https://tarflow.mis.nsysu.edu.tw/login?next=%2Fmain%2F662d88ed73

In [ ]:
#匯入資料集
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/metaData2_news.csv")
# df["artDate"] = pd.to_datetime(df["artDate"])
# df['artDate'] = df['artDate'].dt.date

clear_df = df.copy()

#去除本次不會用到的欄位
drop_cols = ['system_id', 'content']
clear_df.drop(drop_cols, axis = 1, inplace = True)

#選出100比文章資料
sampled_df = clear_df.sample(n=100, random_state=222)

sampled_df

,Unnamed: 0,artUrl,artDate,artTitle,artContent,artCatagory,sentence
768,768,https://www.dcard.tw/f/dressup/p/254969233,2024-03-14 06:48:16,分享紐約穿搭從早秋到冬的章節,分享一些平價穿搭圖片順序從去年九月到二月上衣搭牛仔長褲也很好看裙子在童裝買的這件很適合派對像...,穿搭,分享一些平價穿搭圖片順序從去年九月到二月上衣搭牛仔長褲也很好看裙子在童裝買的這件很適合派對像...
523,523,https://www.dcard.tw/f/graduate_school/p/25484...,2024-03-01 14:19:22,重考心路歷程,第一次認真發文手機排版如果很亂的話請見諒看到一些同學目前放榜的結果好像並不是很理想好像有些同...,研究所,第一次認真發文手機排版如果很亂的話請見諒看到一些同學目前放榜的結果好像並不是很理想好像有些同...
218,218,https://www.dcard.tw/f/dressup/p/254907155,2024-03-07 17:03:57,日常穿搭分享台北日遊穿搭,大家好最近剛好有去台北放鬆幾天逃離一下社畜生活也搭了幾套穿搭想分享一下希望大家會喜歡很少打文...,穿搭,大家好最近剛好有去台北放鬆幾天逃離一下社畜生活也搭了幾套穿搭想分享一下希望大家會喜歡很少打文...
775,775,https://www.dcard.tw/f/dressup/p/254952527,2024-03-12 13:00:12,日常穿搭近期的一些上班穿搭分享,新品輕型牛仔襯衫超寬版牛仔直筒褲輕薄襯衫格紋超寬版直筒褲尼龍襯衫牛仔超寬版直筒褲男裝壓線男裝...,穿搭,新品輕型牛仔襯衫超寬版牛仔直筒褲輕薄襯衫格紋超寬版直筒褲尼龍襯衫牛仔超寬版直筒褲男裝壓線男裝...
349,349,https://www.dcard.tw/f/money/p/254852906,2024-03-02 17:13:52,樂天信貸照會,想請問大家樂天照會會詢問哪些問題呢還有禮拜三開戶和信貸一起申請大概還需要等多久才會打電話照會呢,理財,想請問大家樂天照會會詢問哪些問題呢還有禮拜三開戶和信貸一起申請大概還需要等多久才會打電話照會呢
...,...,...,...,...,...,...,...
1044,1044,https://www.dcard.tw/f/money/p/254854997,2024-03-03 03:42:43,開槓桿買,老公預計在年底有張股票月收萬老婆在年底開槓桿信貸額度未定但利息每個月萬從老公那邊補兩個人在年...,理財,老公預計在年底有張股票月收萬老婆在年底開槓桿信貸額度未定但利息每個月萬從老公那邊補兩個人在年...
1050,1050,https://www.dcard.tw/f/money/p/254834978,2024-03-01 04:00:01,薩爾瓦多總統持有的比特幣均價,中美洲國家薩爾瓦多年月正式宣布將比特幣變成國家的使用法幣還記得當年聽到有把比特幣當法幣的國家...,理財,中美洲國家薩爾瓦多年月正式宣布將比特幣變成國家的使用法幣還記得當年聽到有把比特幣當法幣的國家...
590,590,https://www.dcard.tw/f/graduate_school/p/25490...,2024-03-07 09:51:49,國北幼教研究所,各位好本人目前不是本科但是很喜歡小孩所以目前正在私立幼稚園擔任助理教保員但是嗯薪水太低加上蠻...,研究所,各位好本人目前不是本科但是很喜歡小孩所以目前正在私立幼稚園擔任助理教保員但是嗯薪水太低加上蠻...
705,705,https://www.dcard.tw/f/dressup/p/255003738,2024-03-18 02:53:02,分享上胸不豐滿也能穿辣人的衣服,週末的時候跟男友去逛一間服飾店試穿了下面這件衣服男友說你敢穿這樣出去嗎我說這不就是露上胸的衣...,穿搭,週末的時候跟男友去逛一間服飾店試穿了下面這件衣服男友說你敢穿這樣出去嗎我說這不就是露上胸的衣...


以下以 JSON檔 為例

In [ ]:
class board(BaseModel):
    情緒: str = Field(..., description="判斷整個句子總體的情緒。", enum=["正向", "中性", "負向"])
    個版: str = Field(
        ...,
        description="列出所有句子中提及的情緒字眼及關鍵字,像是穿搭,投資理財,星座,研究所等,並以逗號','分隔。"
    )
    Dcard版種類: str = Field(
        ...,
        description="判斷文章中提到的句子,判斷是屬於什麼Dcard版。",
        enum=["穿搭", "理財", "星座", "研所", "無法判定"],
    )

#Output Parser 目的是要讓 LLM 生成的內容符合特定格式，例如：JSON, list, YAML 等
board_parser = JsonOutputParser(pydantic_object=board)

# 透過 get_format_instructions() 可以看到如何告訴 LLM 生成我們想要的格式的一些指令
board_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"\\u60c5\\u7dd2": {"title": "\\u60c5\\u7dd2", "description": "\\u5224\\u65b7\\u6574\\u500b\\u53e5\\u5b50\\u7e3d\\u9ad4\\u7684\\u60c5\\u7dd2\\u3002", "enum": ["\\u6b63\\u5411", "\\u4e2d\\u6027", "\\u8ca0\\u5411"], "type": "string"}, "\\u500b\\u7248": {"title": "\\u500b\\u7248", "description": "\\u5217\\u51fa\\u6240\\u6709\\u53e5\\u5b50\\u4e2d\\u63d0\\u53ca\\u7684\\u60c5\\u7dd2\\u5b57\\u773c\\u53ca\\u95dc\\u9375\\u5b57,\\u50cf\\u662f\\u7a7f\\u642d,\\u6295\\u8cc7\\u7406\\u8ca1,\\u661f\\u5ea7,\\u7814\\u7a76\\u6240\\u7b49,\\

得到JSON格式的內容後,建立一個_emplate,並告訴他我們需要什麼資料,應輸出什麼資訊,格式等等

In [ ]:
# Set up a parser + inject instructions into the prompt template.
board_parser = JsonOutputParser(pydantic_object=board)\

board_template = """<start_of_turn>user
你是一位社群媒體分析專家，尤其對Dcard有獨到見解,也很擅長看出文章情緒,你將會看Dacrd上對於個版的評論，請抓取出評論中：
1.此評論的情緒，正向或負向或中性
2.有討論到關鍵詞，例如：穿搭,衣服,投資,理財,星座,研究所等
3.個版類型，例如：穿搭版、星座版、研究所版、理財版、無法認定

以下為一些範例：
```
範例1
句子：風格跟以前好像轉變蠻大的～
現在都穿深色系為主 簡單又搭
啊身高體重還是一樣 172/51 🤣
那就開始分享唄！！

LOOK1
這套真的是我的愛⋯黑白配是不會錯的！
背心後面是做挖背 非常涼～😆
版型貼身但能遮副乳 左下有做個小開衩
增加一咪咪設計感！（不過洗後下擺會有些捲邊～

褲子版型非常棒 高個友善！長度很夠><
垂墜的布料讓整體看起來很高級又有質感🥺
第一條用髒我還去重買一條！

抓取結果：{{
    "情緒":"正向",
    "關鍵詞":"背心","褲子","開衩","布料","貼身",
    "個版種類":"穿搭版"
}}

範例2
句子：
白羊座：白羊座面對真愛都是害羞的少女少年。白羊座其實沒有你們想像得大膽直接，他只敢對便當和朋友兇而已！撞見真愛時的他不但沒有膽量，還會變得有點自卑（這點你有看過嗎？）在你面前開始像個上幼稚園的孩子，不敢主動表達，就代表他愛上你了！然而，遇見真愛的白羊，最不勇敢的是，「他真的很沒骨氣的想為你做很多事！」

◇金牛座： 金牛座面對真愛時就是個大管家。金牛座在生活中，不關他的事情他其實不會主動去管！然而遇見真愛時的金牛座，卻開始幻想你家的床單是不是該換天絲絨、你的洗髮精跟護髮乳還夠用嗎？他最不勇敢的地方是：「想打理你的人生」他也很痛恨自己為什麼要幫你想這麼多？他很希望自己還是那頭驕傲的牛！但喜歡就容易從眼角裡面洩出來，不勇敢，又怎樣呢？

◇雙子座：雙子座面對真愛都是嚴肅的精算師。雙子座看似朋友眾多不缺對象的外表底下，撞見真愛時的他反而會收起笑臉，開始變得正經嚴肅，深怕一個表現錯誤，就把你給嚇跑。他最不勇敢的地方是：「想太多」，雙子愛上一個人，最怕的就是自己沒辦法承擔責任！（因為他知道他好奇心旺盛），可是他又很想給你未來，他變得嚴肅並不是因為你做錯什麼，而是他愛上你了。

◇巨蟹座： 巨蟹座面對真愛時情緒化會比過往多120%。巨蟹座真的也很受不了自己這般小劇場（偶爾是歌劇院），所以他會刻意在對方面前表現大方、不在意、沒關係、我沒差，但回到家馬上開始限動發小作文、聽悲傷情歌、傳訊給好友問問題，巨蟹的不勇敢來自於「心口不一」。內心想爆發情感、動作止於禮貌，當你看到一個巨蟹這種雙面表現時，請心疼他：「最近心裡住了誰？」

◇獅子座：獅子座對真愛時會變成對方的粉絲。外界都覺得獅子座是需要別人崇拜他、需要膜拜他的星座！不過獅子座內心很需要被呼呼、需要被M、需要一個讓他崇拜的人。獅子座私下是個心思很細膩的人，他有他自己的理想世界，只是對外他必須是「獅子座」而裝作很強勢。唯有真愛能讓他軟弱。他最不勇敢的是，明明是隻大貓，還是得在你面前裝作是強者獅子。

抓取結果：{{
    "情緒":"中性",
    "關鍵詞":"白羊座","金牛座","雙子座","獅子座","撞見真愛時",“星座”,"心思很細膩",
    "個版種類":"星座版"
}}


範例3
句子：近期因為有一筆資金需求，因本身有協商，條件差的可以，詢問多家代辦基本上都是被婉拒，後面找到「鉅烽金融」他們說可以幫我辦，前面有只跟我說收費標準（以下圖片）

撥款日當天跟我說還要收申覆處理手續費$12000

因為我真的不是很了解
想詢問這樣合理嗎？
抓取結果：{{
    "情緒":"負向",
    "關鍵詞":"資金","金融","撥款日","代辦","手續費",
    "個版種類":"理財版"
}}


範例4
句子：目前大二 因為上學期學期總成績平均85 所以去查了一下關於獎學金申請的資格
發現沒有符合資格並可以申請的獎學金 或是我沒找到（？
很多北部跟宮廟的獎學金是有戶籍地跟科系的限制
松山慈祐宮、竹林山觀音寺⋯⋯等等 我都不能申請
我也查過淡水獎學金 只有看到淡水出生獎學金或是淡大的學生的
但我不是在淡水出生的 條件不符 無法申請🥲
我讀東吳 戶籍在淡水區 非清寒
不是想當伸手牌 是真的找不到才上來問的🥹🧎
國高中也不是在淡水讀書 所以沒有認識戶籍也在淡水的朋友可以討論🥹
雖然成績也不是非常好 但還是想試試看

抓取結果：{{
    "情緒":"負向",
    "關鍵詞":"學期總成績","淡大","獎學金","成績","系的限制",
    "個版種類":"研究所版"
}}

{format_instructions}
請使用繁體中文回答。
除了 JSON 內容外，不要回答任何其他內容

現在請幫我抓取以下句子的資訊：
句子：{query}
抓取結果：<end_of_turn>
<start_of_turn>model
"""

board_prompt = PromptTemplate(
    template=board_template,
    # template="Answer the user query.\n{format_instructions}\nDon't output any other explanation\n{query}\n",
    input_variables=["query"],
    # 將 format_instructions 也輸入到 prompt templete 中
    partial_variables={"format_instructions": board_parser.get_format_instructions()},
)

chain = board_prompt | llm
chain.get_graph().print_ascii()

       +-------------+         
       | PromptInput |         
       +-------------+         
              *                
              *                
              *                
      +----------------+       
      | PromptTemplate |       
      +----------------+       
              *                
              *                
              *                
   +---------------------+     
   | HuggingFaceEndpoint |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| HuggingFaceEndpointOutput |  
+---------------------------+  


手動建立 parse function

In [ ]:
# 從 json 字串中抓取答案
def extract_answer_from_string(string_):
    string_ = string_.replace("\n", "")
    string_ = string_.replace("json", "")
    string_ = string_.replace("    ", "")
    string_ = string_.replace("```", "")
    try:
        # 可以成功轉換
        dict_result = json.loads(string_)
    except:
        print(string_)
        return string_
    return dict_result

# 從 dict 抓取情緒
def get_sentiment(row):
    sentiment = ''
    if isinstance(row, dict):
        if ("情緒" in row.keys()):
            sentiment = row['情緒']
    return sentiment

In [ ]:
llm_result = extract_answer_from_string(chain.invoke({"query": "超喜歡這套薄荷綠的洋裝(●’◡’●)ﾉ裡面搭背心就很日系了這種柔柔的薄荷綠一看就讓人覺得氣質可愛兼具啊(⊃｡•́‿•̀｡)⊃"}))
print(llm_result)
get_sentiment(llm_result)

{"情緒":"正向","關鍵詞":"洋裝","背心","薄荷綠","氣質","可愛","日系","個版種類":"穿搭版"}
{"情緒":"正向","關鍵詞":"洋裝","背心","薄荷綠","氣質","可愛","日系","個版種類":"穿搭版"}


''

以上可以看出回答的內容會照我們剛才template設定的內容,格式回應,且相當準確

In [ ]:
sampled_df['LLM'] = sampled_df.apply(lambda x: extract_answer_from_string(chain.invoke({"query": x['sentence']})), axis=1)
sampled_df

# sampled_df['sentiment'] = sampled_df.apply(lambda x: get_sentiment(x['LLM']), axis=1)
# sampled_df.head()
# sampled_df['sentiment'].value_counts()

{"情緒": "正向","關鍵詞": "裙子", "牛仔長褲", "上衣", "皮褲", "針織上衣", "牛仔靴", "小個子", "白襯衫", "米色", "杏色", "白色", "NGL", " knelt","個版種類": "穿搭版"}
{"情緒":"中性","關鍵詞":"重考研究所","放榜","備取","進度","個版種類":"研究所版"}
{"情緒": "中性","關鍵詞": "西裝外套", "西裝褲", "針織衫", "牛仔褲", "設計", "休閒", "小正式","個版種類": "穿搭版"}
{"情緒":"正向","關鍵詞":"牛仔襯衫","直筒褲","格紋","尼龍襯衫","超寬版","個版種類":"穿搭版"}
{"情緒":"中性","關鍵詞":"樂天照會","禮拜三開戶","信貸","個版種類":"無法認定"}
{"情緒":"負向","關鍵詞":"光信貸","利率","月付","中信","轉貸","個版種類":"理財版"}
{  "情緒": "正向",  "關鍵詞": "化工所", "報考", "成績", "台大", "清大", "成大", "考古題", "題庫班", "工數", "老師", "學長",  "個版種類": "以下文字為個人分享，並非特定版型"}
{"情緒": "中性","關鍵詞": "台銀","黃金存摺","收益","變現","配息","個版種類": "無法認定"}
{"情緒":"正向","關鍵詞":"古娃娃","包","集結","個版種類":"無法認定"}
{"情緒": "中性","關鍵詞": "研究所考試","面談信","放榜","教授","收滿","回簡","狀況","個版種類": "研究所版"}
{"情緒":"中性","關鍵詞":"進貨","貼文","退貨","個版種類":"無法認定"}
{"情緒":"正向","關鍵詞":"夏天","上衣","花紋","魚尾裙","外套","色系","短版","吊帶","小鑽格紋","下擺","個版種類":"穿搭版"}
{"情緒":"中性","關鍵詞":"中央產經所","錄取","正取生會","備取","個版種類":"無法認定"}
{"情緒": "正向","關鍵詞": "洋裝", "灰色", "焦圓圓", "私訊", "買","個版種類": "無法認定"}
{"情緒":"正向","關鍵

,Unnamed: 0,artUrl,artDate,artTitle,artContent,artCatagory,sentence,LLM
768,768,https://www.dcard.tw/f/dressup/p/254969233,2024-03-14 06:48:16,分享紐約穿搭從早秋到冬的章節,分享一些平價穿搭圖片順序從去年九月到二月上衣搭牛仔長褲也很好看裙子在童裝買的這件很適合派對像...,穿搭,分享一些平價穿搭圖片順序從去年九月到二月上衣搭牛仔長褲也很好看裙子在童裝買的這件很適合派對像...,"{""情緒"": ""正向"",""關鍵詞"": ""裙子"", ""牛仔長褲"", ""上衣"", ""皮褲"", ""..."
523,523,https://www.dcard.tw/f/graduate_school/p/25484...,2024-03-01 14:19:22,重考心路歷程,第一次認真發文手機排版如果很亂的話請見諒看到一些同學目前放榜的結果好像並不是很理想好像有些同...,研究所,第一次認真發文手機排版如果很亂的話請見諒看到一些同學目前放榜的結果好像並不是很理想好像有些同...,"{""情緒"":""中性"",""關鍵詞"":""重考研究所"",""放榜"",""備取"",""進度"",""個版種類""..."
218,218,https://www.dcard.tw/f/dressup/p/254907155,2024-03-07 17:03:57,日常穿搭分享台北日遊穿搭,大家好最近剛好有去台北放鬆幾天逃離一下社畜生活也搭了幾套穿搭想分享一下希望大家會喜歡很少打文...,穿搭,大家好最近剛好有去台北放鬆幾天逃離一下社畜生活也搭了幾套穿搭想分享一下希望大家會喜歡很少打文...,"{""情緒"": ""中性"",""關鍵詞"": ""西裝外套"", ""西裝褲"", ""針織衫"", ""牛仔褲""..."
775,775,https://www.dcard.tw/f/dressup/p/254952527,2024-03-12 13:00:12,日常穿搭近期的一些上班穿搭分享,新品輕型牛仔襯衫超寬版牛仔直筒褲輕薄襯衫格紋超寬版直筒褲尼龍襯衫牛仔超寬版直筒褲男裝壓線男裝...,穿搭,新品輕型牛仔襯衫超寬版牛仔直筒褲輕薄襯衫格紋超寬版直筒褲尼龍襯衫牛仔超寬版直筒褲男裝壓線男裝...,"{""情緒"":""正向"",""關鍵詞"":""牛仔襯衫"",""直筒褲"",""格紋"",""尼龍襯衫"",""超寬版..."
349,349,https://www.dcard.tw/f/money/p/254852906,2024-03-02 17:13:52,樂天信貸照會,想請問大家樂天照會會詢問哪些問題呢還有禮拜三開戶和信貸一起申請大概還需要等多久才會打電話照會呢,理財,想請問大家樂天照會會詢問哪些問題呢還有禮拜三開戶和信貸一起申請大概還需要等多久才會打電話照會呢,"{""情緒"":""中性"",""關鍵詞"":""樂天照會"",""禮拜三開戶"",""信貸"",""個版種類"":""無..."
...,...,...,...,...,...,...,...,...
1044,1044,https://www.dcard.tw/f/money/p/254854997,2024-03-03 03:42:43,開槓桿買,老公預計在年底有張股票月收萬老婆在年底開槓桿信貸額度未定但利息每個月萬從老公那邊補兩個人在年...,理財,老公預計在年底有張股票月收萬老婆在年底開槓桿信貸額度未定但利息每個月萬從老公那邊補兩個人在年...,"{""情緒"": ""中性"",""關鍵詞"": ""股票"", ""月收"", ""槓桿信貸"", ""利息"", ""..."
1050,1050,https://www.dcard.tw/f/money/p/254834978,2024-03-01 04:00:01,薩爾瓦多總統持有的比特幣均價,中美洲國家薩爾瓦多年月正式宣布將比特幣變成國家的使用法幣還記得當年聽到有把比特幣當法幣的國家...,理財,中美洲國家薩爾瓦多年月正式宣布將比特幣變成國家的使用法幣還記得當年聽到有把比特幣當法幣的國家...,"{""情緒"": ""正向"",""關鍵詞"": ""薩爾瓦"",""比特幣"",""法幣"",""通膨"",""外國貨幣..."
590,590,https://www.dcard.tw/f/graduate_school/p/25490...,2024-03-07 09:51:49,國北幼教研究所,各位好本人目前不是本科但是很喜歡小孩所以目前正在私立幼稚園擔任助理教保員但是嗯薪水太低加上蠻...,研究所,各位好本人目前不是本科但是很喜歡小孩所以目前正在私立幼稚園擔任助理教保員但是嗯薪水太低加上蠻...,"{""情緒"": ""正向"",""關鍵詞"": ""小孩"",""幼稚園"",""入學"",""研究所"",""研究計畫..."
705,705,https://www.dcard.tw/f/dressup/p/255003738,2024-03-18 02:53:02,分享上胸不豐滿也能穿辣人的衣服,週末的時候跟男友去逛一間服飾店試穿了下面這件衣服男友說你敢穿這樣出去嗎我說這不就是露上胸的衣...,穿搭,週末的時候跟男友去逛一間服飾店試穿了下面這件衣服男友說你敢穿這樣出去嗎我說這不就是露上胸的衣...,"{""情緒"":""正向"",""關鍵詞"":""服飾"",""衣服"",""胸部"",""挖胸上衣"",""高腰亮面絲滑..."


最後我們在sampled_df建立了LLM欄,此欄位包含了情緒,關鍵詞,個版種類,可以輕易看出此篇Dcard文章的情緒值,所屬個版及判斷原因包含的關鍵字。

#5.GPT模型應用
此部分導入gpt-3.5-turbo-1106模型

主要應用為

-將兆豐銀行112年11月13日投資研究週報PDF文檔抓取

-將文檔丟入模型中讓其整理分析

-生成出此文檔可能出現的相關資訊問題
並顯示出答案

-可以Q&A中快速找出文檔內容重點及建立問答集

-同時可做為日後finetune新模型的再訓練資料

-以此參考資料來提升模型答案的精準度

In [ ]:
#連結openai_api_key
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [ ]:
import requests
import json
from pprint import pp

#定義函數
def get_completion(messages, model="gpt-3.5-turbo-1106", temperature=0, max_tokens=2000, format_type=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if format_type:
    payload["response_format"] =  { "type": format_type }

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [ ]:
!pip install langchain

In [ ]:
#安裝兆豐PDF
!wget https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/112/1121113.pdf

--2024-05-20 18:50:52--  https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/112/1121113.pdf
Resolving www.megabank.com.tw (www.megabank.com.tw)... 23.210.106.144
Connecting to www.megabank.com.tw (www.megabank.com.tw)|23.210.106.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999199 (1.9M) [application/pdf]
Saving to: ‘1121113.pdf’

1121113.pdf         100%[===================>]   1.91M  1.73MB/s    in 1.1s    

2024-05-20 18:50:55 (1.73 MB/s) - ‘1121113.pdf’ saved [1999199/1999199]



In [ ]:
from langchain.document_loaders import PyPDFLoader

#讀取檔案
loader = PyPDFLoader("1121113.pdf")
documents = loader.load()

prompt 參考自 https://smith.langchain.com/hub/homanp/question-answer-pair

In [ ]:
pairs = []

for page in documents:
  print(page)
  for i in range(1,2):  # 會跑 1 次
    print(f"------{i}")

    # 資料清理，把沒用的文字去掉
    data = { "context": page.page_content.replace('本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。','') }

    prompt = """You are an AI assistant tasked with generating question and answer pairs for the given context.
Only answer in the format with no other text. Return a question/answer pair as JSON.
請用台灣繁體中文產生問答。

Format:

{
  "question": "string", // relevant question to the context
  "answer": "string" //relevant answer to the question and context
}

Context: """ + data["context"]

    result = get_completion([{"role" : "user", "content": prompt}], temperature=0.2, model="gpt-3.5-turbo-1106", format_type="json_object")
    result2 = json.dumps(result)
    data = json.loads(result2)
    pairs.append(data)
#pairs

page_content='本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。\n本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。1\n財富管理處 投顧小組\n112年11月13日投資研究週報' metadata={'source': '1121113.pdf', 'page': 0}
------1
page_content='本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。2內容大綱\n2\n市場回顧\n2\n3\n41\n聚焦議題\n資產觀點市場焦點' metadata={'source': '1121113.pdf', 'page': 1}
------1
page_content='本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。主要金融市場回顧\n資料來源 :Bloomberg ，2023.11.10 ，投顧小組整理。全球主要股票指數 (%) 全球主要債券指數 (%)\n全球主要商品指數 (%) 全球主要匯率 (%)\n市\n場\n回\n顧\n市\n場\n焦\n點\n聚\n焦\n議\n題\n資\n產\n觀\n點\n股票指數過去1個月\n漲跌幅%過去1年\n漲跌幅%年初迄今\n漲跌幅%\nMSCI AC世界指數 0.44% 9.64% 10.06%\nS&P 500指數 1.31% 11.60% 15.00%\n歐洲道瓊600指數 -2.03% 2.64% 4.34%\nMSCI亞太不含日本指數 -0.62% 5.89% -3.75%\n日經225指數 2.59% 18.66% 24.81%\nMSCI新興市場指數 0.40% 6.60% -0.84%漲跌幅\n週漲跌幅%\n0.53%\n1.31%\n-0.21%\n-0.

In [ ]:
# import ast
# for pair in pairs:
#     data = ast.literal_eval(pair)
#     answer = data["answer"]
#     print(answer)

#將Q&A列出
for pair in pairs:
   #print( "Q: " + pair["question"])
   #print( "A: " + pair["answer"])
   #轉換資料格式
   data = ast.literal_eval(pair)
   print( "Q: " + data["question"])
   print( "A: " + data["answer"])
   #answer = data["answer"]
   #print(answer)
   #print(pair)
   print("-----")

Q: 這份投資研究週報的內容是什麼？
A: 這份投資研究週報是由財富管理處投顧小組在112年11月13日所發布的。
-----
Q: 市場回顧的內容是什麼？
A: 聚焦議題和資產觀點市場焦點
-----
Q: 全球主要股票指數過去1年漲跌幅為多少？
A: MSCI AC世界指數 9.64%, S&P 500指數 11.60%, 歐洲道瓊600指數 2.64%, MSCI亞太不含日本指數 5.89%, 日經225指數 18.66%, MSCI新興市場指數 6.60%
-----
Q: 台灣出口年比數據是多少？
A: 0.61%
-----
Q: 美債殖利率市場關鍵提問
A: 美國聯準會暫停升息，美債殖利率開始下滑，投資人認為12月暫停升息的機率達86%
-----
Q: 美國聯邦基金目標利率是什麼？
A: 美國聯邦基金目標利率是指美國聯邦基金利率的目標範圍，由美國聯邦儲備系統所設定。
-----
Q: 美國消費信心指數下降至多少？
A: 11月密西根大學消費信心降至 60.4
-----
Q: 美國政府關門危機再現對市場有什麼可能影響？
A: 可能影響 :(1)美股為短期干擾。 (2)美債殖利率在財政疑慮下偏升，但景氣降溫使上行壓力受阻。 (3)美元地位受質疑。
-----
Q: 日本薪資成長動能如何？
A: 日本薪資成長動能疲乏
-----
Q: 中國10月進口為多少億美元？
A: 2,183億美元
-----
Q: 中國10月消費者物價指數年增率為多少？
A: -0.2%
-----
Q: 印度在人口方面有什麼優勢？
A: 印度在今年已取代中國成為世界第一人口大國，而且年齡中位數僅 28.7歲，比中國的 38.4歲還年輕10歲，可望帶來強勁的生產力與消費力。
-----
Q: 印度2023年度資本支出規模預估為多少？
A: 印度2023 年度資本支出規模超過 35兆印度盧比 (約4,200億美元 )
-----
Q: 142023 年第四季推薦標的是什麼？
A: 神助攻 ~G.O.D.
-----
Q: 基金配息率是否代表基金報酬率？
A: 基金配息率不代表基金報酬率 ，且過去配息率不代表未來配息率
-----


透過讀取PDF文檔,將資料丟入GPT模型中,分析整理文檔,

生成出相關的問題與答案,以此幫助本小組快速找出此文檔的重點與關注,

也可做為日後提升回答準確度的訓練資料。

In [ ]:
#可將以上Q&A丟入訊息中做為日後finetune的參考
transformed_data = [
  {"messages": [{"role": "system", "content": "You are a helpful chatbot about finance"}, {"role": "user", "content": ["question"]}, {"role": "assistant", "content": data["answer"]}]} for pair in pairs
]

In [ ]:
#將可能做為日後finetune的資料存成sonl格式
file_path = 'training_data.jsonl'
with open(file_path, 'w', encoding='utf-8') as jsonl_file:
    for entry in transformed_data:
        jsonl_file.write(json.dumps(entry) + '\n')

file_path

'training_data.jsonl'

導入PDF文件後,用gpt-3.5-turbo模型呼叫看看回答的內容

In [ ]:
get_completion([{"role" : "user", "content": "日本薪資成長動能如何？"}], temperature=0, model="gpt-3.5-turbo")

'日本的薪資成長動能一直被認為是相對緩慢的。這主要是由於日本長期面臨經濟增長緩慢、通膨率低以及勞動市場結構性問題等因素所導致的。然而，近年來日本政府積極推動經濟改革和勞動市場改革，以促進薪資增長和提高勞動生產力。\n\n其中一個重要的舉措是通過提高最低工資水平，以確保低收入工作者能夠獲得更高的收入。此外，日本政府還推出了一系列措施，包括提高女性就業率、推動青年就業和培訓計劃等，以促進勞動市場的活躍度和競爭力。\n\n雖然這些舉措有助於提高日本的薪資水平，但要實現真正的薪資增長還需要更多的努力。日本企業需要更多地投資於人力資本和技術創新，以提高勞動生產力和競爭力。此外，政府還需要進一步推動結構性改革，以減少勞動市場的僵化和不確定性，從而促進薪資增長和經濟增長。'

In [ ]:
get_completion([{"role" : "user", "content": "112年11月13日投資研究週報是由哪個部門發布的？"}], temperature=0, model="gpt-3.5-turbo")

'這份投資研究週報是由公司的研究部門發布的。'

In [ ]:
get_completion([{"role" : "user", "content": "美國政府關門危機再現的原因是什麼？"}], temperature=0, model="gpt-3.5-turbo")

'美国政府关门危机再现的原因主要是因为国会未能通过一项资金法案来为政府提供资金，导致政府无法正常运作。这种情况通常发生在国会无法就政府预算达成一致意见时，导致政府资金耗尽而不得不关闭部分或全部政府机构和服务。这种情况通常会对国家经济和民众生活造成严重影响。'

In [ ]:
get_completion([{"role" : "user", "content": "基金經理公司以往之經理績效好嗎？"}], temperature=0, model="gpt-3.5-turbo")

'這取決於具體的基金經理公司和其過去的表現。有些基金經理公司可能有出色的績效紀錄，而有些則可能表現一般或不佳。投資者可以通過查看基金經理公司的歷史績效報告和評論來評估其過去的表現。另外，也可以參考專業機構對該公司的評級和建議來了解其績效情況。最重要的是，投資者應該根據自己的投資目標和風險承受能力來選擇適合自己的基金經理公司。'

In [ ]:
get_completion([{"role" : "user", "content": "中國10月進口意外呈正成長多少？"}], temperature=0, model="gpt-3.5-turbo")

'根據中國海關總署的數據，2021年10月，中國進口總值為2.06萬億人民幣，同比增長20.1%。這表明中國的進口意外呈現正成長。'

#6.Retrieve QA (RAG)

*   官網：https://python.langchain.com/v0.1/docs/use_cases/question_answering/quickstart/
*   讀入 PDF 檔

*   切分檔案 chunk

##6.1資料導入

此部分選擇中華郵政悠遊VISA卡的調款約定書的PDF檔做為資料來進行RAG

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/2024_study_group9_4/LLM/data/郵政VISA悠遊金融卡約定條款.pdf")
pages = loader.load()
pages
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    # 設定chunk size
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(pages)
texts[:10]

[Document(page_content='版本： 113.03（EPS00042 ）  頁 1  郵政 VISA悠遊金融卡約定條款  \n   \n郵政金融卡約定條款  \n本契約書由中華郵政股份有限公司 (以下稱貴公司 )提供 \n一、 申請人茲向貴公司申請郵政金融卡（以下稱金融卡）壹枚， 本金融卡具存款、提款、繳稅（費） 、\n密碼變更、查詢餘額、消費扣款等功能，及其因本卡衍生之各項金融服務 (包含但不限於下載「金\n融卡雲支付」或使用金融卡核驗服務 等)。 \n二、 本金融卡不主動提供轉帳功能，如有需要應另行申請。  \n三、 由法定代理人 (或監護人 )代為或允許未成年之申請人訂立本契約時，應同意申請人於領用本卡後\n可使用前述一、二之各項金融服務。  \n雙方嗣後往來願遵守下列各約定條款：  \n第一條  （申請、領取及作廢）  \n申請人申請金融卡，應攜帶國民身分證、儲金簿及原留印鑑至任一郵局（非通儲戶應至 開戶局）辦理，\n並於申請日 5天(工作天 )後，攜帶前述證件於營業時間內至受理郵局領取金融卡及辦理啟用登錄手續 (即\n時發卡者，於申請日辦理啟用及領取手續 )。如自申請日起逾 4個月仍未領取者，貴公司得將金融卡逕行\n繳銷作廢(如欲重新申請，須酌收工本費 )，並依重新申請新卡手續辦理。前述期間貴公司得視需要隨時調\n整，申請人同意由貴公司於營業場所或網站上公告其內容，且不經通知即生效力。  \n第二條  （密碼變更）  \n申請人於貴公司 窗口申辦金融卡 時，須自行設定 6至12位數晶片密碼，如欲變更晶 片密碼者，得利用貴\n公司自動櫃員機（或網路 ATM）自行更改密碼，其次數不受限制。  \n第三條  （存款金額之限制 ） \n申請人使用金融卡於自動櫃員機存入本人或他人帳戶之現金，每次及每日存款總額及手續費，以貴公司網\n站公告為主。另申請人同意存入他人帳戶如 須繳納手續費，應由存入金額扣除。  \n第四條  （提款及轉帳金額之限制）  \n申請人使用金融卡於貴公司自動櫃員機提款及轉帳時，其上限如下：  \n一、 提款每次最高限額為新臺幣 6萬元，每日最高限額（與跨行提款合計）為新臺幣 15萬元。  \n二、 約定帳戶轉帳每次最高限額為新臺幣 100萬元，每日最高限額（含跨行、非跨行）為新臺幣 100\n萬元。  \n三、 非約定帳戶轉帳

- 將文檔轉換為 embedding
- 存入 vectorDB

In [ ]:
#模型導入後將句子轉成embedding
embedding_function = SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding")

#取出向量
vectorstore = Chroma.from_documents(documents=texts, embedding=embedding_function)

retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

測試找尋相關文檔

In [ ]:
# 詢問問題
query = "提款卡提款每日最高限額？"
docs = vectorstore.similarity_search(query)

# print results
print(docs[0].page_content)

申請人使用金融卡於貴公司自動櫃員機提款及轉帳時，其上限如下：  
一、 提款每次最高限額為新臺幣 6萬元，每日最高限額（與跨行提款合計）為新臺幣 15萬元。  
二、 約定帳戶轉帳每次最高限額為新臺幣 100萬元，每日最高限額（含跨行、非跨行）為新臺幣 100
萬元。  
三、 非約定帳戶轉帳每次最高限額為新臺幣 3萬元，每日最高限額（含跨行、非跨行）為新臺幣 3
萬元。  
第五條  （跨行提款及轉帳金額之限制）  
申請人使用金融卡在參加金融資訊系統跨行連線金融單位設置自動化服務設備提款及轉帳時，上限如下：  
一、 跨行提款每次最高限額為新臺幣 2萬元，每日最高限額 （與非跨行提款合計） 為新臺幣 15萬元。  
二、 約定帳戶轉帳每次最高限額為新臺幣 100萬元，每日最高限額（含跨行、非跨行）為新臺幣 100
萬元。  
三、 非約定帳戶轉帳每次最高限額為新臺幣 3萬元，每日最高限額（含跨行、非跨行） 為新臺幣 3
萬元。  
第六條  （消費扣款）  
一、 申請人於申請金融卡時，金融卡自動預設啟用消費扣款功能。申請人如欲終止該功能者，應向貴
公司提出申請終止該功能，經貴公司作業完成後，始生終止效力。申請人嗣後仍可再向貴公司提
出申請啟用該功能。  
二、 申請人憑金融卡及密碼於郵局 (限實體卡片 )或消費扣款特約商店進行消費扣 款交易時，消費金額
逕自帳戶扣款，且 每次最高限額為新臺幣 10萬元，每日最高限額為新臺幣 20萬元(與轉帳併計100萬元)。倘申請人申請具感應功能之金融卡且消費金額於新臺幣 3千元以下者，部分郵局及
特約商店得免插卡輸密碼，即可以感應式交易結帳。  
三、 申請人連續進行感應式交易付款金額累計逾新臺幣 3千元時，應以插卡輸密碼方式進行交易，始
得繼續使用感應式交易，其後進行感應式交易累計再逾新臺幣 3千元時亦同。  
四、 申請人明確瞭解消費扣款交易與現金交易並無不同，如與特約商店發生消費爭議，皆應向特約商
店尋求解決，不得以此作為向貴公司請求返還帳款之依據。如對相關帳款有疑義時，應於消費扣
款日起90天內向貴公司提出（書面查詢） ，逾期則推定貴公司帳載事項無誤。  
第七條  （跨境電子支付）  
一、 使用跨境電子支付服務時，申請人須 成年及啟用金融卡非 約定轉帳功能， 並配合進行身分確認作
業。


但以上回答結果內容有些多,但顧客可能只需要簡單結果,只想知道想要的答案而已

##6.2 RAG prompt 與 chain

故此部分定義 RAG prompt 與 chain,同時建立回答更精準,合適的問答機器人

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_prompt_templete = """<start_of_turn>user
"角色": "郵局VISA悠遊卡諮詢機器人",
"服務內容":
你是一個熟讀郵局VISA悠遊卡的稽核機器人，關於VISA與悠遊卡約定條款的問題你都可以依照以下文件中的規定來回答學生。
現在給予以下相關文件, 你將會基於以下 [文件][文件] 中的相關文件與 [回答規則][回答規則] 中的回答規則來回答顧客的問題。
\n
[文件]
{context}
[文件]
\n
[回答規則]
只依據 [文件][文件] 中提供的資訊來回答，並且遵照下列的幾點規則：
- 不撒謊或幻想 [文件][文件] 中未明確提供的答案
- 如果不確定答案或答案未明確包含在 [文件][文件] 中，請回答：“我很抱歉，我不知道如何提供幫助。”
- 回答簡短、相關且簡潔的答案
- 如果有條項問題,如“第三條”,請將條文列出
- 內容中若有"\n",請去除
- 回答字數請少於 150 個字
- 請在 [結果]: 之後回答你的答案
[回答規則]

<</SYS>>
現在，基於上述的文件與規則，回答以下 () 中的問題\n[顧客]:({question})\n<end_of_turn>
<start_of_turn>model[結果]:"""

rag_prompt = PromptTemplate.from_template(rag_prompt_templete)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)
rag_chain.get_graph().print_ascii()

              +---------------------------------+           
              | Parallel<context,question>Input |           
              +---------------------------------+           
                    ****                ****                
                 ***                        ***             
               **                              ***          
+----------------------+                          **        
| VectorStoreRetriever |                           *        
+----------------------+                           *        
            *                                      *        
            *                                      *        
            *                                      *        
+---------------------+                     +-------------+ 
| Lambda(format_docs) |                     | Passthrough | 
+---------------------+                     +-------------+ 
                    ****                ****                
                        

In [ ]:
rag_chain.invoke("VISA悠遊卡每日,每月刷卡額度應為多少？")

' VISA悠遊卡每日刷卡額度不得超過新臺幣 2 萬元，每月刷卡額度不得超過新臺幣 10 萬元。'

In [ ]:
rag_chain.invoke("VISA悠遊卡遺失,被竊該如何處理")

' 根據第十四條的規定，持卡人應即時以電話或其他方式通知貴公司辦理掛失停用手續。並繳交掛失補發手續費每卡新臺幣 100元。但如貴公司認為有必要時，得通知持卡人，要求於受通知日起 3日內向當地警察機關報案並檢具報案證明文件補行通知貴公司。'

In [ ]:
rag_chain.invoke("VISA悠遊卡如何補發新卡流程？")

' VISA悠遊卡發生遺失時，可向貴公司申請補發新卡。補發新卡之悠遊卡儲值金餘額為零，舊卡之悠遊卡儲值金餘額將由貴公司於收到卡片後辦理「餘額轉置」作業。'

透過以上將templete進行RAG並定義 RAG prompt 與 chain能發現LLM模型不但能回答更精準，內容也簡短許多，更能符合顧客所需要的答案。